In [2]:
import gym
import math
import time
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
np.random.seed(0)

In [3]:
env = gym.make('FrozenLake-v0')

In [4]:
?env.env

Type:        FrozenLakeEnv
String form: <FrozenLakeEnv<FrozenLake-v0>>
File:        c:\users\taha\anaconda3\lib\site-packages\gym\envs\toy_text\frozen_lake.py
Docstring:  
Winter is here. You and your friends were tossing around a frisbee at the
park when you made a wild throw that left the frisbee out in the middle of
the lake. The water is mostly frozen, but there are a few holes where the
ice has melted. If you step into one of those holes, you'll fall into the
freezing water. At this time, there's an international frisbee shortage, so
it's absolutely imperative that you navigate across the lake and retrieve
the disc. However, the ice is slippery, so you won't always move in the
direction you intend.
The surface is described using a grid like the following

    SFFF
    FHFH
    FFFH
    HFFG

S : starting point, safe
F : frozen surface, safe
H : hole, fall to your doom
G : goal, where the frisbee is located

The episode ends when you reach the goal or fall in a hole.
You receive a 

In [ ]:
for _ in range(3):
    env.reset()
    for _ in range(100):
        env.step(env.action_space.sample())
        env.render()
env.close()

In [22]:
Q_table = np.zeros((16,) + (env.action_space.n,))
Q_table.shape

(16, 4)

In [60]:
tic = time.time()
Q_table = np.zeros((16,) + (env.action_space.n,))
reward_list = []
plot_reward = []
total_reward = 0
consec = 0
success = False
for i in range(1, 100001):
    lr = max(0.01, min(1.0, 1.0 - math.log10((i + 1) / 25)))
    epsilon =  max(0.05, min(1, 1.0 - math.log10((i  + 1) / 25)))
    current = env.reset()
    done = False
    while not done:
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q_table[current])
        observation, reward, done, info = env.step(action) 
        next_state = observation
        Q_table[current][action] += lr*(reward + np.max(Q_table[next_state]) - Q_table[current][action])
        current = next_state
        total_reward += reward
        if done:
            final_reward = reward
    if final_reward == 1:
        consec += 1
    else:
        consec = 0
    if consec == 100:
        toc = time.time()
        success = True
        print(f'Took {toc-tic:.2f} seconds and succeeded in {i} episodes')
        break 
    total_reward = 0
toc = time.time()
if not success:
    print(f'Took {toc-tic:.2f} seconds and failed')
print(Q_table)

Took 67.68 seconds and failed
[[0.76978161 0.73190827 0.72759758 0.73120632]
 [0.37280803 0.34757597 0.28426743 0.6551533 ]
 [0.51062117 0.30275957 0.32160538 0.34558829]
 [0.21757013 0.05268623 0.03320303 0.06713249]
 [0.77007717 0.51421304 0.52191354 0.53765541]
 [0.         0.         0.         0.        ]
 [0.37652058 0.1698534  0.3429371  0.14332201]
 [0.         0.         0.         0.        ]
 [0.55737331 0.50249115 0.50326395 0.77069244]
 [0.5125774  0.77191449 0.46605895 0.47601717]
 [0.71412928 0.54010665 0.4329502  0.36295429]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.53220515 0.59193911 0.84701425 0.56130968]
 [0.83358377 0.9302524  0.86195797 0.84471407]
 [0.         0.         0.         0.        ]]


In [61]:
Q_table_100k = Q_table

In [59]:
tic = time.time()
Q_table = np.zeros((16,) + (env.action_space.n,))
policy = np.zeros(16)
reward_list = []
plot_reward = []
total_reward = 0
consec = 0
success = False
for i in range(1, 30001):
    lr = max(0.01, min(1.0, 1.0 - math.log10((i + 1) / 25)))
    epsilon =  max(0.05, min(1, 1.0 - math.log10((i  + 1) / 25)))
    current = env.reset()
    done = False
    while not done:
        action = int(policy[current])
        observation, reward, done, info = env.step(action) 
        next_state = observation
        Q_table[current][action] += lr*(reward + Q_table[next_state][int(policy[next_state])] - Q_table[current][action])

        if np.random.random() <= epsilon:
            policy[next_state] = env.action_space.sample()
        else:
            policy[next_state] = np.argmax(Q_table[next_state])
        current = next_state
        total_reward += reward
        if done:
            final_reward = reward
    if final_reward == 1:
        consec += 1
    else:
        consec = 0
    if consec == 100:
        toc = time.time()
        success = True
        print(f'Took {toc-tic:.2f} seconds and succeeded in {i} episodes')
        break 
    total_reward = 0
toc = time.time()
if not success:
    print(f'Took {toc-tic:.2f} seconds and failed')
print(Q_table)

Took 17.98 seconds and failed
[[5.07962058e-01 3.39101535e-01 3.34459703e-01 1.63899391e-01]
 [1.90775401e-01 8.01555902e-02 3.85038264e-02 9.91121436e-02]
 [2.07395178e-01 5.61332939e-02 2.80806712e-02 3.48458174e-02]
 [8.62732111e-05 0.00000000e+00 0.00000000e+00 1.19378486e-02]
 [5.23713133e-01 2.72853070e-01 3.38677816e-01 2.54106682e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.53620385e-01 1.12412000e-01 1.61403316e-01 5.02370876e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.56091629e-01 2.65116481e-01 3.40409251e-01 5.58795911e-01]
 [4.18043587e-01 6.09951034e-01 3.72485511e-01 3.40674928e-01]
 [5.51787993e-01 4.02717260e-01 2.64991131e-01 2.69115599e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.13003060e-01 3.33324066e-01 7.51965232e-01 3.99030127e-01]
 [4.71045661e-01 8.62752909e-01 6.79287163e-01 6.86458370e-01]
 [0.00000000e+00 0.000000